https://developers.google.com/gmail/api/quickstart/python

In [ ]:
!hostname

# install modules

In [ ]:
!pip install --upgrade google-api-python-client

In [ ]:
!pip install --upgrade oauth2client

In [ ]:
from __future__ import print_function
from googleapiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools

# authenticate our token

The following code cell intentionally prevents the rest of the notebook from automatically executing

In [ ]:
stop -- action needed

the script below must be run as a .py file in order to launch your browser and authenticate

The .py file contains

If the error message is

`raise HttpAccessTokenRefreshError(error_msg, status=resp.status)
oauth2client.client.HttpAccessTokenRefreshError: invalid_grant: Bad Request`
        
then delete "token.json"

# now that we've authenticated with our token, we can send email
https://developers.google.com/gmail/api/guides/sending

## message text needs to be base64 encoded

https://en.wikipedia.org/wiki/Base64

"Base64 is a group of binary-to-text encoding schemes that represent binary data in an ASCII string format by translating it into a radix-64 representation."

Why do we use base64 for SMTP? Answer: SMTP is so old that the number of characters is stricted. To expand the character set, base64 encode. https://www.quora.com/Why-are-MIME-attachments-encoded-in-base64 and https://superuser.com/questions/402193/why-is-base64-needed-aka-why-cant-i-just-email-a-binary-file

In [ ]:
import base64
base64.b64encode(bytes('your string', 'utf-8'))

a function that creates a base64 encoded email

In [ ]:
#https://docs.python.org/3/library/email.mime.html
from email.mime.text import MIMEText
from email import encoders
    
def create_message(sender, to, subject, message_text):
    """Create a message for an email.

    Args:
    sender: Email address of the sender.
    to: Email address of the receiver.
    subject: The subject of the email message.
    message_text: The text of the email message.

    Returns:
    An object containing a base64url encoded email object.
    """
    message = MIMEText(message_text)
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    
    # https://docs.python.org/3/library/base64.html
    return {'raw': base64.urlsafe_b64encode(message.as_string().encode('utf-8')).decode("utf-8")}
    # https://stackoverflow.com/questions/606191/convert-bytes-to-a-string
    
    #return {'raw': base64.urlsafe_b64encode(bytes("None",'utf-8'))}
    
    # https://docs.python.org/3.4/library/email-examples.html
    #return {'raw': encoders.encode_base64(message.as_string())} 

In [ ]:
msg=create_message(sender="benpayne@umbc.edu", 
                   to="ben.is.located@gmail.com", 
                   subject="a subjct", 
                   message_text="msg text from Python")
msg

In [ ]:
from googleapiclient import errors

def send_message(service, user_id, message):
    """Send an email message.

  Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    message: Message to be sent.

  Returns:
    Sent Message.
    """
    try:
        message = (service.users().messages().send(userId=user_id, body=message)
                   .execute())
        print ('Message Id: %s' % message['id'])
        return message
    # https://stackoverflow.com/questions/42541857/error-using-gmail-api-tuto-using-python-3-except-errors-httperror-error
    except errors.HttpError as error:
        print ('An error occurred: %s' % error)

In [ ]:
store = file.Storage('token.json')
creds = store.get()
service = build('gmail', 'v1', http=creds.authorize(Http()))

The following also works

# with attachment

In [ ]:
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
import mimetypes
import os

In [ ]:
# https://developers.google.com/gmail/api/guides/sending#creating_messages_with_attachments
def create_message_with_attachment(
    sender, to, subject, message_text, file):
    """Create a message for an email.

  Args:
    sender: Email address of the sender.
    to: Email address of the receiver.
    subject: The subject of the email message.
    message_text: The text of the email message.
    file: The path to the file to be attached.

  Returns:
    An object containing a base64url encoded email object.
    """
    message = MIMEMultipart()
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject

    msg = MIMEText(message_text)
    message.attach(msg)

    content_type, encoding = mimetypes.guess_type(file)

    if content_type is None or encoding is not None:
        content_type = 'application/octet-stream'
    main_type, sub_type = content_type.split('/', 1)
    if main_type == 'text':
        fp = open(file, 'rb')
        msg = MIMEText(fp.read(), _subtype=sub_type)
        fp.close()
    elif main_type == 'image':
        fp = open(file, 'rb')
        msg = MIMEImage(fp.read(), _subtype=sub_type)
        fp.close()
    elif main_type == 'audio':
        fp = open(file, 'rb')
        msg = MIMEAudio(fp.read(), _subtype=sub_type)
        fp.close()
    else:
        fp = open(file, 'rb')
        msg = MIMEBase(main_type, sub_type)
        msg.set_payload(fp.read())
        fp.close()
    filename = os.path.basename(file)
    msg.add_header('Content-Disposition', 'attachment', filename=filename)
    message.attach(msg)

#    return {'raw': base64.urlsafe_b64encode(message.as_string())}
    return {'raw': base64.urlsafe_b64encode(message.as_string().encode('utf-8')).decode("utf-8")}

# rename files

In [1]:
import glob
import hashlib
import time
import shutil
import random

In [2]:
list_of_grp1_files = glob.glob('grp1/*.ipynb')

In [4]:
len(list_of_grp1_files)

25

In [5]:
list_of_grp2_files = list_of_grp1_files#glob.glob('grp/*.ipynb')

In [6]:
#len(list_of_dice_files)

### demo of hash from string

In [7]:
hashlib.md5(b"Nobody inspects the spammish repetition").hexdigest()

'bb649c83dd1ea5c9d9dec9a18df0ffe9'

### we'll need a list of the student IDs later

In [8]:
list_of_student_ids = []
for file_name in list_of_grp1_files:
    user_name = file_name.split('/')[-1].split('.')[0]
    list_of_student_ids.append(user_name)

In [9]:
list_of_student_ids

['pb70051',
 'jconroy1',
 'cj86416',
 'hlaper1',
 'jliang5',
 'ti83170',
 'tev1',
 'vjyesta1',
 'am27',
 'gmoummi1',
 'ro30897',
 'dbrown8',
 'parthms1',
 'pmourya1',
 'hq16977',
 'anudeek1',
 'tris1',
 'wfrenc1',
 'kw35573',
 'n98',
 'tugcee1',
 'zt71691',
 'chandus1',
 'bsama1',
 'vl29331']

In [ ]:
store = file.Storage('token.json')
creds = store.get()
service = build('gmail', 'v1', http=creds.authorize(Http()))

In [ ]:
for which_list in [list_of_fake_files, list_of_dice_files]:
    list_of_students = list_of_student_ids.copy()

    for file_name in which_list:
        #print('file name =',file_name)
        #print('len of list of students',len(list_of_students))
        folder_name = file_name.split('/')[0]
        user_name = file_name.split('/')[-1].split('.')[0] # who wrote the file?
        name_hash = hashlib.md5(str.encode(user_name)).hexdigest() # hash of student name
        file_hash_name = folder_name + '/' + name_hash + '.ipynb'
        #print('file_name=',file_name,'; file_hash_name=',file_hash_name)
        shutil.copy(file_name, file_hash_name)
        # find another student to send this file to
        send_to = user_name
        while send_to == user_name:
            send_to = random.choice(list_of_students)
        list_of_students.remove(send_to)
        #print(list_of_students)
        print(folder_name,'author:',user_name,':', name_hash,'; reviewer =', send_to+'@umbc.edu')
    
        msg = create_message_with_attachment(
            'ben@com', # sender; doesn't seem to show up in actual email
#            'ben.is.located@gmail.com', # to - for testing 
            send_to+'@umbc.edu',  # MASS MAIL - destination student
            '601 notebook for review : '+folder_name+" "+name_hash, # subject
#            'Task: Review this notebook.\n'+
            'Task: attempt to run this notebook.\n'+
            'Reply to this email by 9pm on April 8 if you have feedback for the notebook author.\n\n'+
            'What changes (if any) to the code did you need to make?\n'+
#            'For example, is 
            'Was the documentation sufficient?\n'+
            'Were you able to get the code to run?\n'+
            'If no, where did you get stuck?\n'+
            'Did you encounter any errors?\n'+
            'Was the output correct?\n'+
#            'Are you able to understand the author\'s intent?\n'+
            'What suggestions do you have for the author?\n\n'+
            'Do not sign the reply email with your name.\n'+
            'Your comments should be in the email body. \n'+
            'Do not include any attachments in the reply email.', # body
            file_hash_name)
#        if send_to=='pb70051':
        send_message(service, "me", msg)  # send all the mails
        print('removing',file_hash_name)
        os.remove(file_hash_name)